In [1]:
import pandas as pd
import requests
import json 
import time
from tqdm import tqdm
from datetime import datetime, timezone, timedelta
import re

In [3]:
token = ''

In [5]:
getGroupID_url = f'https://api.vk.com/method/groups.getById?group_id=ria&access_token={token}&v=5.131'
resp = requests.get(getGroupID_url).json()
group_id = f'-{resp['response'][0]['id']}'
group_name = resp['response'][0]['name']
print(f'Сообщество: {group_name}. ID: {group_id}')

Сообщество: РИА Новости. ID: -15755094


In [7]:
url = f"https://api.vk.com/method/wall.get?owner_id={group_id}&count=1&access_token={token}&v=5.131"
response = requests.get(url).json()
total_posts = response['response']['count']
print(f'Всего постов в {group_name} {total_posts}')

Всего постов в РИА Новости 248672


In [7]:
def get_post_by_id (post_id):
    url_id = f"https://api.vk.com/method/wall.getById?posts={group_id}_{post_id}&access_token={token}&v=5.131"
    response1 = requests.get(url_id).json()
    return response1['response']

In [11]:
def date_to_unix (date: str): # ex: "2024-01-01 00:00:00"
    '''перевод даты в unix формат'''
    dt = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = dt.replace(tzinfo=timezone(timedelta(hours=3)))
    timestamp = int(dt.timestamp())
    return timestamp

In [13]:
def date_from_unix (date: int): 
    '''перевод даты в привычный формат'''
    return datetime.fromtimestamp(date)

# Сбор постов

***Структура: id поста, текст, количество комментариев, дата***

In [ ]:
start_date = 1704056400 # 1.01.2024 00:00:00
end_date = 1706734799 # 31.01.2024 23:59:59
all_posts = {key: [] for key in ['id', 'Posts', 'NComments', 'Date']} #сюда тащим все-все посты
flag = False 
reposts = 0

for i in tqdm(range(0, total_posts, 100)):
    if flag:
        break
    url = f'https://api.vk.com/method/wall.get?owner_id={group_id}&count=100&offset={i}&access_token={token}&v=5.131'
    response = requests.get(url).json()
    if 'response' not in response or 'items' not in response['response']:
        continue
    posts = response['response']['items']
    for post in posts:
        date = post["date"]
        if date < start_date:   #отбор по дате
            flag = True
            break
        elif date > end_date:
            continue
             
        else:
            if 'copy_history' in post: # пропускаем репосты
                reposts += 1
                continue  
                
            text = post['text']
            if text == '' or any(map(str.isalpha, text)) == False:
                if 'attachments' not in post or 'link' not in post['attachments'][0]:
                    continue
                text = post['attachments'][0]['link']['title']                     
                                                         
            id = post['id']
            
            try:
                ncomm = post['comments']['count']
            except KeyError:
                ncomm = -1
            
            if start_date <= date <= end_date:
                all_posts['id'].append(id)
                all_posts['Posts'].append(text)
                all_posts['NComments'].append(ncomm)
                all_posts['Date'].append(date)
    
        time.sleep(0.3)
        
    time.sleep(0.5)

# Сбор комментариев

***Структура: id поста, id комментария, текст комментария (помечен Comm) или ответа на него***

In [349]:
january = pd.read_excel('1january.xlsx')

In [351]:
#нам нужны только id поста и количество комментариев, поэтому создадим вспомогательный дф
january = january[['id', 'NComments']]
january

,id,NComments
0,44734671,17
1,44734493,21
2,44734225,49
3,44734058,49
4,44733908,60
...,...,...
1164,44452087,143
1165,44451796,393
1166,44451751,76
1167,44451386,587


In [357]:
all_comments = {key: [] for key in ['PostID', 'CommentID', 'CommentsAndReplies']}
count = 0
key_errors = []
for i in tqdm(range(len(january))):
    post_id = january['id'][i]
    n_comm = january['NComments'][i]
    for j in range (0, n_comm, 100):
        url_comm = f'https://api.vk.com/method/wall.getComments?v=5.131&owner_id={group_id}&post_id={post_id}&count=100&offset={j}&thread_items_count=10&access_token={token}'
        response_comm = requests.get(url_comm).json()
        
        if 'response' not in response_comm:
            continue
            key_errors.append(post_id)
        for comm in response_comm['response']['items']:
            comment = comm['text']
            comm_id = comm['id']
            all_comments['PostID'].append(post_id)
            all_comments['CommentID'].append(comm_id)
            all_comments['CommentsAndReplies'].append(f'Comm: {comment}')
            count+=1
            if comm['thread']['count'] > 0: 
                for t in range(0, comm['thread']['count'], 100):
                    url_replies = f"https://api.vk.com/method/wall.getComments?v=5.131&owner_id={group_id}&post_id={post_id}&comment_id={comm_id}&count=100&offset={t}&access_token={token}"
                    replies = requests.get(url_replies).json()
                    if 'response' not in replies:
                        continue
                        key_errors.append((post_id, comm_id))
                        
                    for one in replies['response']['items']:
                        reply = one['text']
                        all_comments['PostID'].append(post_id)
                        all_comments['CommentID'].append(comm_id)
                        all_comments['CommentsAndReplies'].append(reply)
                        count +=1
                time.sleep(0.2)
            time.sleep(0.2)
        time.sleep(0.2)
    time.sleep(0.3) 

100%|████████████████████████████████████████████████████████████████████████████| 1169/1169 [6:15:11<00:00, 19.26s/it]


In [359]:
comments = pd.DataFrame(all_comments)

In [341]:
comments = comments.dropna()

In [361]:
comments

,PostID,CommentID,CommentsAndReplies
0,44734671,44734701,Comm:
1,44734671,44734701,"[id833417405|Будем], так переезжай в Москву. Ч..."
2,44734671,44734701,"[id93364207|Константин], всем россиянам в Моск..."
3,44734671,44734777,Comm: Разберитесь с кидаловом на счетчиках вна...
4,44734671,44734777,"[id407713064|Андрей], а что мешает самому уста..."
...,...,...,...
129294,44451363,44453893,Comm: З новим роком..
129295,44451363,44453928,"Comm: Да неужели, Белгород теперь в списках. В..."
129296,44451363,44455565,Comm: С новым годом товарищи😘
129297,44451363,44455859,Comm: С новым годом2024 Россия желаю всем поб...


In [363]:
comments.to_excel('comm_january.xlsx', index = False)

In [365]:
comments.to_json('comm_january.json', orient='records', indent=4, force_ascii=False)